# Démonstration de coreferee sur du français

## Charger un modèle
Example avec le modèle large fr_core_news_lg

In [1]:
import spacy, coreferee

In [2]:
nlp = spacy.load("fr_core_news_lg")
nlp.add_pipe("coreferee")
nlp.pipe_names

['tok2vec',
 'morphologizer',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'coreferee']

## Identifier les chaines de coréférence

In [3]:
text = "Même si elle était très occupée par son travail, Julie en avait marre. Alors, elle et son mari décidèrent qu'ils avaient besoin de vacances. Ils allèrent en Espagne car ils adoraient le pays."
doc = nlp(text)
doc


Même si elle était très occupée par son travail, Julie en avait marre. Alors, elle et son mari décidèrent qu'ils avaient besoin de vacances. Ils allèrent en Espagne car ils adoraient le pays.

### Afficher les chaines

In [4]:
doc._.coref_chains.print()

0: elle(2), son(7), Julie(10), elle(17), son(19)
1: travail(8), en(11)
2: [elle(17); mari(20)], ils(23), Ils(29), ils(34)
3: Espagne(32), pays(37)


### Parcourir les chaines

In [5]:
for chain in doc._.coref_chains:
    print("La chaine numéro", chain.index, "contient les mentions suivantes :")
    for mention in chain:
        print("\t", mention.pretty_representation, "composée des tokens numéro :", mention.token_indexes)

La chaine numéro 0 contient les mentions suivantes :
	 elle(2) composée des tokens numéro : [2]
	 son(7) composée des tokens numéro : [7]
	 Julie(10) composée des tokens numéro : [10]
	 elle(17) composée des tokens numéro : [17]
	 son(19) composée des tokens numéro : [19]
La chaine numéro 1 contient les mentions suivantes :
	 travail(8) composée des tokens numéro : [8]
	 en(11) composée des tokens numéro : [11]
La chaine numéro 2 contient les mentions suivantes :
	 [elle(17); mari(20)] composée des tokens numéro : [17, 20]
	 ils(23) composée des tokens numéro : [23]
	 Ils(29) composée des tokens numéro : [29]
	 ils(34) composée des tokens numéro : [34]
La chaine numéro 3 contient les mentions suivantes :
	 Espagne(32) composée des tokens numéro : [32]
	 pays(37) composée des tokens numéro : [37]


### Résoudre la coréférence pour un token donné (par exemple un pronom):
Obtenir la mention la plus représentative de ce token

In [5]:
print("Le pronom «", doc[2], "» désigne", doc._.coref_chains.resolve(doc[2]))

Le pronom « elle » désigne [Julie]


## Construire les mentions complètes
Obtenir les syntagmes (nominaux ou pronominaux) entiers à partir des têtes

In [7]:
from build_mentions import build_mention, create_mentions

In [8]:
doc = nlp("Les enfants de l'école primaire rentrent chez leurs parents")
doc._.coref_chains.print()

0: enfants(1), leurs(8)


### Retrouver le syntagme d'une mention

In [9]:
print("Le syntagme de la tête «", doc[1], "» est «", build_mention([doc[1]], nlp), "»")

Le syntagme de la tête « enfants » est « Les enfants de l'école primaire »


### Obtenir les syntagmes et leur numéro de chaine pour tout le document
Même les singletons ! 

In [10]:
new_mentions = create_mentions(doc, nlp, add_singletons=True)
new_mentions

{Les enfants de l'école primaire: 0,
 leurs: 0,
 l'école primaire: 1,
 leurs parents: 2}

In [11]:
from build_mentions import make_new_chains

make_new_chains(new_mentions)


{0: [Les enfants de l'école primaire, leurs],
 1: [l'école primaire],
 2: [leurs parents]}

## Exemples réels

### Article de presse
Source : https://www.lemonde.fr/election-presidentielle-2022/article/2022/03/04/eric-zemmour-condamne-pour-la-quatrieme-fois_6116185_6059010.html

In [12]:
doc = nlp('''Eric Zemmour condamné pour la quatrième fois
Le candidat d’extrême droite a été condamné à une amende pour « contrefaçon de droit d’auteur » par le tribunal de Paris. Il avait utilisé des extraits vidéo sans autorisation dans son clip de déclaration de candidature.

Eric Zemmour a été de nouveau condamné, vendredi 4 mars, mais cette fois pour « contrefaçon de droit d’auteur », après la diffusion de son clip de candidature, le 30 novembre 2021, où il avait utilisé nombre d’extraits de films sans aucune sorte d’autorisation.

Le candidat d’extrême droite à l’élection présidentielle, son parti Reconquête ! et François Miramont, alors responsable de l’association Les Amis de Zemmour, se voient condamnés à verser 5 000 euros à chacune des onze parties civiles et 10 000 euros de frais de justice, soit un total de 165 000 euros. Ils disposent d’une semaine pour couper les séquences litigieuses, sous peine d’une astreinte de 1 500 euros par jour de retard – ce qui revient à supprimer la fameuse vidéo.

« Il n’est plus temps de réformer la France, mais de la sauver. C’est pourquoi j’ai décidé de me présenter à l’élection présidentielle », avait déclaré l’ancien chroniqueur du Figaro dans une vidéo crépusculaire de dix minutes. Il lisait un texte, dans une sombre bibliothèque et devant un micro très Radio-Londres, entrecoupé d’images d’archives et de sa campagne, sur fond du 2e mouvement de la 7e Symphonie de Beethoven.
''')
doc

Eric Zemmour condamné pour la quatrième fois
Le candidat d’extrême droite a été condamné à une amende pour « contrefaçon de droit d’auteur » par le tribunal de Paris. Il avait utilisé des extraits vidéo sans autorisation dans son clip de déclaration de candidature.

Eric Zemmour a été de nouveau condamné, vendredi 4 mars, mais cette fois pour « contrefaçon de droit d’auteur », après la diffusion de son clip de candidature, le 30 novembre 2021, où il avait utilisé nombre d’extraits de films sans aucune sorte d’autorisation.

Le candidat d’extrême droite à l’élection présidentielle, son parti Reconquête ! et François Miramont, alors responsable de l’association Les Amis de Zemmour, se voient condamnés à verser 5 000 euros à chacune des onze parties civiles et 10 000 euros de frais de justice, soit un total de 165 000 euros. Ils disposent d’une semaine pour couper les séquences litigieuses, sous peine d’une astreinte de 1 500 euros par jour de retard – ce qui revient à supprimer la fameus

In [13]:
new_mentions = create_mentions(doc, nlp)
make_new_chains(new_mentions)

{0: [Eric Zemmour condamné pour la quatrième fois,
  Le candidat d’extrême droite,
  Il,
  son,
  Eric Zemmour,
  son,
  il,
  Le candidat d’extrême droite à l’élection présidentielle, son parti Reconquête ! et François Miramont, alors responsable de l’association,
  son,
  Zemmour],
 1: [la quatrième fois, cette fois],
 2: [François Miramont, responsable de l’association],
 3: [Les Amis de Zemmour, Ils],
 4: [la France, la],
 5: [l’ancien chroniqueur du Figaro, Il, sa]}

### Avis d'acheteur

In [14]:

doc = nlp('''👀 Acheté pour ma nièce pour son anniversaire. Sa mère et moi ne voulions pas que ce soit son premier téléphone coûté très cher. Alors après quelques recherches, j'ai sélectionné celui-ci.

📳 Avant de le lui offrir, j'ai fait un test et j'ai essayé d'évaluer le potentiel, et comment j'ai voulu m'adapter aux besoins de notre jeune ado j'ai immédiatement remarqué la haute qualité des images.
📷 Les 4 caméras permettent de combler les écarts qualitatifs; les photos présentées dans cet avis ont été prises au cours de la journée ( matin par temps gris ) et donc je ne peux rien dire sur les photos de nuit; néanmoins l'autofocus du smartphone a beaucoup de difficulté à esquisser un objectif correct à certaines distances; la caméra réagit bien même dans des conditions de faible luminosité ( à l'ombre ) tout en gardant un excellent niveau de détail. Bonne sélection exacte du foyer en gros plan; couleurs parfaites.

📌 En ce qui a trait aux performances du smartphone; les 4 Go de RAM permettent une navigation en douceur et un jeu défilant supporté par un processeur Unisoc T310.

👍Un écran plein Waterdrop 6.517" HD peut contenir la taille du smartphone lui-même; avec également comme sécurité, vous pouvez définir les empreintes digitales, je trouve que pour le prix de vente par rapport à la qualité de l'appareil il a un pourcentage très bénéfique, parfait pour ma nièce. Elle était enchantée de son cadeau pour son âge.
Moi à mon époque, j'utilisais déjà une ficelle et 2 pots de yogourt pour communiquer entre amis.

J'espère que mes observations vous seront utiles et vous aideront à faire un choix positif ou négatif, mais je suis toujours disponible pour répondre à vos questions.
⬇⬇️ Merci de cliquer sur le bouton Vote utile''')
doc

👀 Acheté pour ma nièce pour son anniversaire. Sa mère et moi ne voulions pas que ce soit son premier téléphone coûté très cher. Alors après quelques recherches, j'ai sélectionné celui-ci.

📳 Avant de le lui offrir, j'ai fait un test et j'ai essayé d'évaluer le potentiel, et comment j'ai voulu m'adapter aux besoins de notre jeune ado j'ai immédiatement remarqué la haute qualité des images.
📷 Les 4 caméras permettent de combler les écarts qualitatifs; les photos présentées dans cet avis ont été prises au cours de la journée ( matin par temps gris ) et donc je ne peux rien dire sur les photos de nuit; néanmoins l'autofocus du smartphone a beaucoup de difficulté à esquisser un objectif correct à certaines distances; la caméra réagit bien même dans des conditions de faible luminosité ( à l'ombre ) tout en gardant un excellent niveau de détail. Bonne sélection exacte du foyer en gros plan; couleurs parfaites.

📌 En ce qui a trait aux performances du smartphone; les 4 Go de RAM permettent une

In [15]:
new_mentions = create_mentions(doc, nlp)
new_chains = make_new_chains(new_mentions)
new_chains

{0: [ma nièce pour son anniversaire, son, Sa, son],
 1: [son premier téléphone, celui-ci, le lui],
 2: [la journée, matin],
 3: [des conditions de faible luminosité ( à l'ombre, en],
 4: [lui-même, il],
 5: [ma nièce, Elle, son, son]}

### Tweets

In [67]:
doc = nlp('''Solaire et rayonnant, @Cyprien_Zeni est à l'image de son île, La Réunion ☀
Il nous entraîne avec son titre "Ma famille"...  le choisirez vous pour nous représenter pour cette édition ?

#EurovisionFrance #Eurovision2022'''.replace("@","").replace("#",""))
print(doc, "\n---------------")
make_new_chains(create_mentions(doc, nlp))

Solaire et rayonnant, Cyprien_Zeni est à l'image de son île, La Réunion 
Il nous entraîne avec son titre "Ma famille"...  le choisirez vous pour nous représenter pour cette édition ?

#EurovisionFrance #Eurovision2022 
---------------


{0: [Cyprien_Zeni, son, Il, son, le]}

In [19]:
doc = nlp('''Yseult. Aucune modestie. Aucune classe. 
Je voulais qu’elle représente la France, mais si c’est pour entendre "I AM not agree" en direct quand elle aura 4 points no thanks. Mes oreilles d’être humain ET de prof d’anglais saignent. #cvqd #EurovisionFrance''')
print(doc, "\n---------------")
make_new_chains(create_mentions(doc, nlp))

Yseult. Aucune modestie. Aucune classe. 
Je voulais qu’elle représente la France, mais si c’est pour entendre "I AM not agree" en direct quand elle aura 4 points no thanks. Mes oreilles d’être humain ET de prof d’anglais saignent. #cvqd #EurovisionFrance 
---------------


{0: [Yseult, elle, elle]}

In [59]:
doc = nlp('''Tadej Pogacar. Cet homme est la future légende du cyclisme moderne, tout simplement. 

J’ai posé mes couilles avec 
@GaltierFabien
 et les bugattis aussi chez 
@BDPetrelli
. 

Continuer à nous sous-estimer. 

PS : Demain j’ai rendez-vous avec la gloire ''')
print(doc, "\n-------------")
make_new_chains(create_mentions(doc, nlp))

Tadej Pogacar. Cet homme est la future légende du cyclisme moderne, tout simplement. 

J’ai posé mes couilles avec 
@GaltierFabien
 et les bugattis aussi chez 
@BDPetrelli
. 

Continuer à nous sous-estimer. 

PS : Demain j’ai rendez-vous avec la gloire  
-------------


{0: [Tadej Pogacar, Cet homme, la future légende du cyclisme moderne]}